In [55]:
import pandas as pd

In [56]:
DATA = "./local_path.csv" # original dataset should have 9089 images

In [57]:
df = pd.read_csv(DATA)

In [58]:
len(df)

9089

In [69]:
# just did hand cleaning of cover images, now run create_path_table again...

In [70]:
df = pd.read_csv(DATA)

In [71]:
len(df)

8946

In [75]:
# now a bunch of extra cleaning, hand removing the white pages, hand removing more covers
# which are sometimes at the bottom of the folder, a few doubles, some book covers they advert...

In [76]:
# now another run of create_path_table...

In [74]:
df = pd.read_csv(DATA)
len(df)

8423

In [81]:
df.reset_index(drop=True,inplace=True)

In [77]:
# Now to remove other doubles occurring within a single issue

In [85]:
i_strings = []

for i in range(len(df)):
    i_string = df.local_path.loc[i][45:].split("/")[0]
    i_strings.append(i_string)

In [86]:
df['iss'] = i_strings

In [88]:
# We'll try to find them first by size

In [ ]:
# eliminate DS_Store files...

In [95]:
for i in range(len(df)):
    s = df.local_path.loc[i]
    if "DS_Store" in s:
        df = df.drop(i)

In [96]:
df.reset_index(drop=True,inplace=True)

In [98]:
size = []

for item in df.local_path:
    img = Image.open(item)
    size.append(img.size)
    
df['size'] = size

In [100]:
iss_names = list(set(list(df.iss)))

In [108]:
counter = 0
dbls = []

for iss in iss_names:
    tmp = df[df.iss==iss]
    tmp_sizes = list(tmp['size'])
    if len(tmp_sizes) > len(set(tmp_sizes)):
        counter+=1
        dbls.append(iss)
        
print counter

90


In [115]:
tmp = df[df.iss.isin(dbls)] # which rows are in issues with size-doubles

In [116]:
len(tmp)

5385

In [117]:
# now mark which sizes per issue have size-doubles

In [121]:
dbl_images = []

for dbl in dbls:
    iss_subset = tmp[tmp.iss==dbl]
    iss_subset.reset_index(drop=True,inplace=True)
    size_bag = list(iss_subset['size'])
    for i in range(len(iss_subset)):
        size = iss_subset['size'].loc[i]
        if size_bag.count(size) > 1:
            dbl_images.append(iss_subset.local_path.loc[i])

In [126]:
double = []
for i in range(len(df)):
    filestring = df.local_path.loc[i]
    if filestring in dbl_images:
        double.append("yes")
    else:
        double.append("no")

In [127]:
df['double'] = double

In [128]:
tmp = df[df.double=="yes"]

In [129]:
len(tmp)

1167

In [130]:
tmp.reset_index(drop=True,inplace=True)

In [131]:
tmp.sort("local_path",inplace=True)

/Library/Python/2.7/site-packages/pandas/core/frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)


In [136]:
tmp['basename'] = tmp.local_path.apply(os.path.basename)
tmp = tmp.sort(["iss","size"])

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [137]:
tmp.to_csv("./doubles.csv",index=False) # just in case things get ugly

In [138]:
# holy crap. That's over. Ok now another path_table...

In [139]:
df = pd.read_csv(DATA)

In [146]:
len(df)

7595

In [ ]:
import os

In [148]:
for i in range(len(df)):
    s = df.local_path.loc[i]
    if "DS_Store" in s:
        os.remove(s)
        df = df.drop(i)
        
df.reset_index(drop=True,inplace=True)

In [ ]:
# well ok one more run of create_path_table, since I've renamed the images folder too...

In [153]:
df = pd.read_csv(DATA)

In [154]:
len(df)

7429

In [155]:
df.to_csv("./local_path_savepoint.csv",index=False)

In [189]:
# so now we have savepoint, and I've moved the cleaned folder to the flash drive
# we'll keep cleaning, but at least we won't lose all our work if it goes wrong

In [141]:
# So we've removed all within-issue doubles, all ads, all covers, all white pages, all crazy shaped
# Also any that appeared to be chopped into pieces

In [142]:
# Now just for kicks, let's visualize these to make sure we've done right:

In [165]:
from PIL import Image, ImageDraw
from skimage import color
from skimage.io import imread
import numpy as np

In [158]:
vals = []
n_channels = []

for file in df.local_path:    
    try:
        img = imread(file)
        channels = len(img.shape)
        n_channels.append(channels)
        img_gray = color.rgb2gray(img)

        # because the bw are [0,255] and the color are [0,1]
        if channels == 2:
            val_mean = np.mean(img_gray) / 255    
        else:
            val_mean = np.mean(img_gray)

        vals.append(val_mean)
    
    except Exception as e:
        vals.append("err")
        n_channels.append("err")
        print file, e
    
df['val'] = vals
df['n_channels'] = n_channels

/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1973.6.issue-4/11.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1973.6.issue-4/11.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/10.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/10.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/11.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/11.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/12.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/12.jpg"
Please see documentation at: h

In [161]:
len(df[df.n_channels=="err"])

785

In [162]:
tmp = df[df.val!="err"]

In [169]:
tmp.sort("val",inplace=True)

In [170]:
tmp.reset_index(drop=True,inplace=True)

In [171]:
import math
side = int(math.ceil( np.sqrt(len(tmp)) ))
thumb_side = 128
px_w = side * thumb_side
px_h = side * thumb_side

In [172]:
canvas = Image.new('RGB',(px_w, px_h),(50,50,50))
x,y = range(side) * side, np.repeat(range(side),side)
tmp['x'] = x[:len(tmp)]
tmp['y'] = y[:len(tmp)]

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [173]:
len(tmp)

6644

In [174]:
for i in range(len(tmp)):
    im = Image.open(tmp.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    x_coord = tmp['x'].loc[i] * thumb_side
    y_coord = tmp['y'].loc[i] * thumb_side
    
    pos_i = (5,5)
    draw = ImageDraw.Draw(im)
    
    if i < 3322:
        draw.text(pos_i, str(i), fill="white")
    else:
        draw.text(pos_i, str(i), fill="black")
    
    canvas.paste(im,(x_coord,y_coord))
    
canvas.save("./CLEAN.png")

In [176]:
# ok some doubles and some super long and thin or wide and short ones...we will clean by hand, of course

In [207]:
tmp.local_path.loc[563]

'/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2000.33.issue-5/64.jpg'

In [208]:
# Ok so there are lots of these little strip images, which are probably an artifact
# of the extraction process, so let's mechanically eliminate some of these:

In [210]:
sizes = []

for file in df.local_path:
    
    try:
        img = Image.open(file)
        sizes.append(img.size)
    except Exception as e:
        sizes.append("err")
        print e
        
df['size'] = sizes

[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1999.32.issue-5/114.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2000.33.issue-5/25.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2000.33.issue-5/64.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38.issue-1/21.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38.issue-1/3.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38.issue-1/35.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38.issue-4/12.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38.issue-4/26.jpg'
[Errno 2] No such file or directory: '/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.2005.38

In [214]:
ratio = []

for i in range(len(df)):
    sz = df['size'].loc[i]
    
    if sz != "err":
        w = sz[0]
        h = sz[1]
        if w > h:
            ratio.append(float(w) / h)
        elif h > w:
            ratio.append(float(h) / w)
        elif w==h:
            ratio.append(1)
            
    else:
        ratio.append("err")
        
df['ratio'] = ratio

In [221]:
iss_img = []

for i in range(len(df)):
    s = df.local_path.loc[i][51:]
    iss_img.append(s)

df['iss_img'] = iss_img

In [225]:
tmp = df[df.ratio!="err"].sort("ratio",ascending=False)

In [235]:
len(tmp[tmp.ratio > 3])

252

In [236]:
tmp = tmp[tmp.ratio > 3] # we'll stipulate that any ratio above 3 is junk

In [238]:
for file in tmp.local_path:
    os.remove(file)

In [ ]:
# now we want to find images with very small sides
# The idea here is: I will thumbnail the visualization to 128px
# If I do, I want still to be able to read even those approaching ratio 3
# So I need to be able to see a third of x. So I decide x=128.

In [241]:
small = []

for i in range(len(df)):
    sz = df['size'].loc[i]
    if sz!="err":
        w = sz[0]
        h = sz[1]
        
        if w < 128 or h < 128:
            small.append("small")
        else:
            small.append("regular")
            
    else:
        small.append("err")
        
df['small'] = small

In [242]:
len(df[df.small=="small"])

127

In [243]:
tmp = df[df.small == "small"]

In [244]:
len(tmp)

127

In [246]:
for file in tmp.local_path:
    if os.path.isfile(file):
        os.remove(file)
    else:
        print "already deleted"

already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted
already deleted


In [247]:
# Ok now another elimination of DS_Store, and another Create Path Table

In [248]:
df = pd.read_csv(DATA)

In [249]:
for i in range(len(df)):
    s = df.local_path.loc[i]
    if "DS_Store" in s:
        os.remove(s)
        df = df.drop(i)
        
df.reset_index(drop=True,inplace=True)

In [250]:
len(df)

7072

In [251]:
vals = []
n_channels = []

for file in df.local_path:    
    try:
        img = imread(file)
        channels = len(img.shape)
        n_channels.append(channels)
        img_gray = color.rgb2gray(img)

        # because the bw are [0,255] and the color are [0,1]
        if channels == 2:
            val_mean = np.mean(img_gray) / 255    
        else:
            val_mean = np.mean(img_gray)

        vals.append(val_mean)
    
    except Exception as e:
        vals.append("err")
        n_channels.append("err")
        print file, e
    
df['val'] = vals
df['n_channels'] = n_channels

/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1973.6.issue-4/11.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1973.6.issue-4/11.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/10.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/10.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/11.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/11.jpg"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/12.jpg Could not load "/Users/damoncrockett/Desktop/Leo/Images_CLEAN/leon.1975.8.issue-2/12.jpg"
Please see documentation at: h

In [252]:
len(df)

7072

In [253]:
len(df[df.val=="err"])

774

In [254]:
# Ok we're getting this error, I think, because some of these are jp2's
# even though they aren't labeled that way

In [278]:
os.mkdir("/Users/damoncrockett/Desktop/Leo/jp2/")

In [261]:
tmp = df[df.val=="err"]

In [262]:
import shutil

In [264]:
tmp.reset_index(drop=True,inplace=True)

In [273]:
ls = []

for i in range(len(tmp)):
    l = tmp.local_path.loc[i][51:].split("/")[0].split(".")
    ls.append(l)

In [274]:
tmp['ls'] = ls

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [276]:
tmp['basename'] = tmp.local_path.apply(os.path.basename)

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [284]:
BASE = "/Users/damoncrockett/Desktop/Leo/jp2/"

targets = []

for i in range(len(tmp)):
    l = tmp.ls.loc[i]
    b = tmp.basename.loc[i]
    s = BASE + str(l[0]) + "_" + str(l[1]) + "_" + str(l[2]) + "_" + b
    targets.append(s)
    
tmp['targets'] = targets

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [285]:
for i in range(len(tmp)):
    source = tmp.local_path.loc[i]
    target = tmp.targets.loc[i]
    shutil.copy(source,target)

In [286]:
# Okay now I've 'converted' these 'jp2s' to jpg; let's see if it worked

In [287]:
import glob

In [300]:
counter=0
for file in glob.glob(os.path.join("/Users/damoncrockett/Desktop/Leo/jp2/","*.jpeg")):
    imread(file)
    counter+=1
print counter

774


In [292]:
# No errors! Okay but now we have to save back to the original file
# We gotta change file extension to jpeg though

In [308]:
tmp.targets.loc[0].rstrip("g") + "eg"

'/Users/damoncrockett/Desktop/Leo/jp2/1973_6_issue-4_11.jpeg'

In [309]:
tmp['targets_jpeg'] = tmp.targets.map(lambda x: x.rstrip("g") + "eg")

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [311]:
tmp.targets_jpeg.loc[0]

'/Users/damoncrockett/Desktop/Leo/jp2/1973_6_issue-4_11.jpeg'

In [312]:
for i in range(len(tmp)):
    target = tmp.local_path.loc[i]
    source = tmp.targets_jpeg.loc[i]
    shutil.copy(source,target)

In [314]:
# okay gotta run create_path_table again

In [315]:
df = pd.read_csv(DATA)

In [320]:
# okay now let's try to extract val

In [316]:
vals = []
n_channels = []

for file in df.local_path:    
    try:
        img = imread(file)
        channels = len(img.shape)
        n_channels.append(channels)
        img_gray = color.rgb2gray(img)

        # because the bw are [0,255] and the color are [0,1]
        if channels == 2:
            val_mean = np.mean(img_gray) / 255    
        else:
            val_mean = np.mean(img_gray)

        vals.append(val_mean)
    
    except Exception as e:
        vals.append("err")
        n_channels.append("err")
        print file, e
    
df['val'] = vals
df['n_channels'] = n_channels

In [318]:
len(df)

7072

In [319]:
df.to_csv("./local_path_savepoint_2_val_channels.csv",index=False)

In [ ]:
# Done!